In [ ]:
# Full combined Jupyter notebook cell for Lighthouse Macro charts
# Includes FRED and BLS API data fetch, plotting, and watermarking

from fredapi import Fred
import requests
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

# API Keys
fred_api_key = '6dcc7a0d790cdcc28c1f751420ee9d27'
bls_api_key = 'e83e702d7e244961951cf9026d1ae437'

# Initialize FRED API
fred = Fred(api_key=fred_api_key)

# Load watermark image (ensure watermark image filename is correct in notebook environment)
watermark = Image.open('image.jpeg').convert("RGBA")

def add_watermark(ax, watermark, alpha=0.15, scale=0.25):
    fig = ax.get_figure()
    fig.canvas.draw()
    bbox = fig.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig_w, fig_h = bbox.width * fig.dpi, bbox.height * fig.dpi
    
    # Resize watermark maintaining aspect ratio
    wm_w, wm_h = watermark.size
    new_size = (int(fig_w*scale), int(wm_h*(fig_w*scale/wm_w)))
    watermark_resized = watermark.resize(new_size, Image.ANTIALIAS)
    
    # Position watermark in bottom right corner with 30 px margin
    margin_x = int(fig_w) - watermark_resized.size[0] - 30
    margin_y = int(fig_h) - watermark_resized.size[1] - 30
    
    # Create an overlay with transparency
    plt_img = Image.new("RGBA", (int(fig_w), int(fig_h)))
    plt_img.paste(watermark_resized, (margin_x, margin_y), watermark_resized)
    plt_img.putalpha(int(255 * alpha))
    
    fig.figimage(plt_img, xo=0, yo=0, zorder=3, alpha=alpha)

### FRED CHARTS ###

# Chart 1: Labor Force Participation Rate
lfpr = fred.get_series('CIVPART', observation_start='2010-01-01', observation_end='2025-08-31')
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(lfpr)
ax.set_title("Labor Force Participation Rate (2010-2025)")
ax.set_xlabel('Date')
ax.set_ylabel('Participation Rate (%)')
ax.grid(True)
add_watermark(ax, watermark)
plt.show()

# Chart 2: Job Openings vs. Unemployed Persons
job_openings = fred.get_series('JTSJOL', observation_start='2019-01-01', observation_end='2025-07-31')
unemployed = fred.get_series('UNEMPLOY', observation_start='2019-01-01', observation_end='2025-07-31')

fig, ax = plt.subplots(figsize=(10,6))
ax.plot(job_openings, label='Job Openings (Millions)')
ax.plot(unemployed, label='Unemployed Persons (Millions)')
ax.set_title('Job Openings vs. Unemployed Persons (2019-2025)')
ax.set_xlabel('Date')
ax.set_ylabel('Millions of Persons')
ax.legend()
ax.grid(True)
add_watermark(ax, watermark)
plt.show()

# Chart 3: Employment Cost Index vs. Consumer Price Index YoY % Change
eci = fred.get_series('ECIALLC', observation_start='2018-01-01', observation_end='2025-06-30')
cpi = fred.get_series('CPIAUCSL', observation_start='2018-01-01', observation_end='2025-06-30')

eci_yoy = eci.pct_change(periods=12) * 100
cpi_yoy = cpi.pct_change(periods=12) * 100

fig, ax = plt.subplots(figsize=(10,6))
ax.plot(eci_yoy, label='ECI YoY %')
ax.plot(cpi_yoy, label='CPI YoY %')
ax.set_title('Employment Cost Index vs. Consumer Price Index (YoY % Change)')
ax.set_xlabel('Date')
ax.set_ylabel('Year-over-Year % Change')
ax.legend()
ax.grid(True)
add_watermark(ax, watermark)
plt.show()

### BLS HELPERS AND CHARTS ###

def fetch_bls_series(series_ids, start_year=2020, end_year=2025):
    headers = {'Content-Type': 'application/json'}
    data = {
        "seriesid": series_ids,
        "startyear": str(start_year),
        "endyear": str(end_year),
        "registrationKey": bls_api_key
    }
    response = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', json=data, headers=headers)
    json_data = response.json()
    
    all_data = []
    for series in json_data['Results']['series']:
        series_id = series['seriesID']
        for item in series['data']:
            if item['period'].startswith('M'):
                all_data.append({
                    'series_id': series_id,
                    'year': int(item['year']),
                    'month': int(item['period'][1:]),
                    'value': float(item['value'])
                })
    return pd.DataFrame(all_data)

# Chart 4: Nonfarm Payrolls by Sector (CES Major Sectors)
sector_series_ids = [
    'CES0500000001', # Total Nonfarm
    'CES6500000001', # Health Care and Social Assistance
    'CES3000000001', # Manufacturing
    'CES9092000001', # Government
    'CES2000000001', # Construction
]

ces_data = fetch_bls_series(sector_series_ids)
ces_data['date'] = pd.to_datetime(ces_data[['year', 'month']].assign(day=1))

fig, ax = plt.subplots(figsize=(12,7))
for series_id in sector_series_ids:
    subset = ces_data[ces_data['series_id'] == series_id]
    ax.plot(subset['date'], subset['value'], label=series_id)

ax.set_title("Nonfarm Payrolls by Sector (2020-2025)")
ax.set_xlabel("Date")
ax.set_ylabel("Employment Level (Thousands)")
ax.legend()
ax.grid(True)
add_watermark(ax, watermark)
plt.show()

# Chart 5: Job Openings Rate by State - California (06) and New York (36)
jolts_state_series_ids = [
    'JLU060000000000003', # CA Job Openings Rate
    'JLU360000000000003', # NY Job Openings Rate
]

jolts_data = fetch_bls_series(jolts_state_series_ids, start_year=2024, end_year=2025)
jolts_data['date'] = pd.to_datetime(jolts_data[['year', 'month']].assign(day=1))

fig, ax = plt.subplots(figsize=(10,6))
for series_id in jolts_state_series_ids:
    subset = jolts_data[jolts_data['series_id'] == series_id]
    ax.plot(subset['date'], subset['value'], label=series_id)

ax.set_title('Job Openings Rate by State (California vs New York)')
ax.set_xlabel('Date')
ax.set_ylabel('Job Openings Rate (%)')
ax.legend()
ax.grid(True)
add_watermark(ax, watermark)
plt.show()

# Chart 6: Projected Job Growth by Sector
# Download CSV from BLS Employment Projections page and load it manually.
# Example (commented since file not available):
'''
import pandas as pd
proj = pd.read_csv('bls_employment_projections_2023_2033.csv')

fig, ax = plt.subplots(figsize=(12,7))
ax.bar(proj['Sector'], proj['Projected Growth'])
ax.set_title("BLS Projected Job Growth by Sector (2023-2033)")
ax.set_xlabel("Sector")
ax.set_ylabel("Projected Job Growth")
plt.xticks(rotation=45, ha='right')
add_watermark(ax, watermark)
plt.show()
'''